In [220]:
import os
import cv2
import numpy as np
import pandas as pd
import re
import random
from shapely.geometry import box, Polygon
from matplotlib import pyplot as plt
#{"None":0,"zh_char":1,"zh_str":2,"en":3,"zh_en":4}

In [221]:
dataset_path = "/home/mark/Desktop/py3.7/cv/datasets/ai_cup_f/"

In [241]:
#read csv
df = pd.read_csv(dataset_path+"lsvt.csv",encoding="utf8")
df.head()

,imgs,pt1,pt2,pt3,pt4,pt5,pt6,pt7,pt8,labels
0,gt_1234,45,272,215,273,212,296,45,290,一站式购物中心
1,gt_1234,33,325,204,328,202,363,31,357,母婴百汇
2,gt_1234,25,359,71,364,70,385,25,382,TEL:
3,gt_1234,72,362,197,365,197,388,72,384,8168886
4,gt_1234,5,394,212,409,209,435,7,423,###


In [242]:
df.isna().sum()

imgs      0
pt1       0
pt2       0
pt3       0
pt4       0
pt5       0
pt6       0
pt7       0
pt8       0
labels    6
dtype: int64

In [243]:
df.labels = df.labels.fillna("###")

In [244]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382606 entries, 0 to 382605
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   imgs    382606 non-null  object
 1   pt1     382606 non-null  int64 
 2   pt2     382606 non-null  int64 
 3   pt3     382606 non-null  int64 
 4   pt4     382606 non-null  int64 
 5   pt5     382606 non-null  int64 
 6   pt6     382606 non-null  int64 
 7   pt7     382606 non-null  int64 
 8   pt8     382606 non-null  int64 
 9   labels  382606 non-null  object
dtypes: int64(8), object(2)
memory usage: 29.2+ MB


In [296]:
def quadril_overlap(poly1,poly2):
    '''
    from shapely.geometry import box, Polygon
    given two polygon points[[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
    calculates the overlap percetage.
    '''
    polygon1_shape = Polygon(poly1)
    polygon2_shape = Polygon(poly2)
    # Calculate Intersection
    polygon_intersection = polygon1_shape.intersection(polygon2_shape).area
    small_box = min(polygon1_shape.area,polygon2_shape.area)
    if polygon_intersection >= small_box*0.5:#若交集區大於最小框的50%，則視為子框(重疊)
        if polygon1_shape.area>polygon2_shape.area:
            return 1
        else:
            return 2
    return 0
def chunks(lst, n):
    ""
    "Yield successive n-sized chunks from lst."
    ""
    for i in range(0, len(lst), n):
        yield lst[i: i + n]

In [265]:
df[df['labels']=="###"]

,imgs,pt1,pt2,pt3,pt4,pt5,pt6,pt7,pt8,labels


In [246]:
df = df[df["labels"]!="###"]#去除###的框

In [297]:
#input df,img_set,bounding_condition="s","l"
def find_bounding_box(df):
    img_set = df.imgs.unique()
    cond=2
    re_box=[]
    for img in img_set:#依序讀入每張圖片
        #print(img)
        img_buf = df[df["imgs"]==img]
        selected_box=[]
        for box in img_buf.iterrows():#一個一個box去看
            #print(selected_box)
            box_pt = list(chunks(box[1][1:9].values,2))
            if not Polygon(box_pt).is_valid:
                continue
            if len(selected_box):
                #print(box_pt)
                score = 0
                for s_box in (selected_box):#若與selected box 重疊則判斷大小，取大的
                    #print(img_buf.loc[12])
                    s_box_pt = list(chunks(img_buf.loc[s_box][1:9].values,2))
                    score_buf = quadril_overlap(s_box_pt,box_pt)#若兩poly重疊回傳且1大:s_box,2大:box, 否則 0
                    score+=score_buf
                    if score_buf:#重疊
                        if score_buf == cond:#1:s_box>box 2:box>s_box
                            selected_box = [box[0] if x==s_box else x for x in selected_box]#取代s_box                        
                if score==0 :#未重疊
                    selected_box.append(box[0])
            else:
                selected_box.append(box[0])
            #print(selected_box)
        re_box.append(list(set(selected_box)))
    return [item for sublist in re_box for item in sublist]

In [298]:
df.head()

,imgs,pt1,pt2,pt3,pt4,pt5,pt6,pt7,pt8,labels
0,gt_1234,45,272,215,273,212,296,45,290,一站式购物中心
1,gt_1234,33,325,204,328,202,363,31,357,母婴百汇
2,gt_1234,25,359,71,364,70,385,25,382,TEL:
3,gt_1234,72,362,197,365,197,388,72,384,8168886
5,gt_1234,649,317,665,317,662,331,648,331,方


In [299]:
bounding_max = find_bounding_box(df)

In [300]:
len(bounding_max)

241855

In [301]:
df_max_bound = df.loc[bounding_max]

In [302]:
#df_max_bound.to_csv(dataset_path+"lsvt_max_bound.csv",encoding="utf",index = False)

In [290]:
Polygon([[45,272],[215,273],[212,296],[45,290]]).is_valid

True